# Lab 1
### Jason McDonald, Miguel Bonilla, Zachary Bunn

## AirBnB Listings for Los Angeles Area

In [5]:
## load pandas and import dataset from GitHub
import pandas as pd

df = pd.read_csv("https://media.githubusercontent.com/media/boneeyah/DS7331_Group/main/Data_Files/airbnb_los_angeles.csv")

In [6]:
df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,45392,https://www.airbnb.com/rooms/45392,20220606014052,2022-06-06,Cute Home in Mount Washington,<b>The space</b><br />Cute house in Mount Wash...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,201514,https://www.airbnb.com/users/show/201514,...,4.96,4.58,4.85,NaN,f,1,0,1,0,0.20
1,45417,https://www.airbnb.com/rooms/45417,20220606014052,2022-06-06,Silver Lake Cottage Oasis- Private Terrace- Th...,"Ideal for an extended stay, our Cottage perfec...",Walk just a couple of blocks to reach all the ...,https://a0.muscache.com/pictures/monet/Select-...,50231247,https://www.airbnb.com/users/show/50231247,...,4.98,4.91,4.82,NaN,f,4,4,0,0,1.09
2,5728,https://www.airbnb.com/rooms/5728,20220606014052,2022-06-06,Tiny Home in Artistic Oasis near Venice and LAX,Our home is located near Venice Beach without ...,Our neighborhood is very quiet and save. There...,https://a0.muscache.com/pictures/7a29d275-f293...,9171,https://www.airbnb.com/users/show/9171,...,4.92,4.80,4.70,HSR19-002149,f,3,0,3,0,2.00
3,5729,https://www.airbnb.com/rooms/5729,20220606014052,2022-06-06,Zen Room with Floating Bed near Venice and LAX,Our home is located near Venice Beach without ...,Our neighborhood is very quiet and save. There...,https://a0.muscache.com/pictures/f48e3ea8-2075...,9171,https://www.airbnb.com/users/show/9171,...,4.89,4.77,4.71,HSR19-002149,f,3,0,3,0,1.52
4,109,https://www.airbnb.com/rooms/109,20220606014052,2022-06-06,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...",NaN,https://a0.muscache.com/pictures/4321499/1da98...,521,https://www.airbnb.com/users/show/521,...,4.00,5.00,4.00,NaN,f,1,1,0,0,0.02


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            42041 non-null  int64  
 1   listing_url                                   42041 non-null  object 
 2   scrape_id                                     42041 non-null  int64  
 3   last_scraped                                  42041 non-null  object 
 4   name                                          42039 non-null  object 
 5   description                                   41102 non-null  object 
 6   neighborhood_overview                         25110 non-null  object 
 7   picture_url                                   42041 non-null  object 
 8   host_id                                       42041 non-null  int64  
 9   host_url                                      42041 non-null 

In [8]:
print(df.room_type.unique())
df.property_type.unique()

## not sure if we should keep both, they're sort of related, but sufficiently different

['Private room' 'Entire home/apt' 'Shared room' 'Hotel room']


array(['Private room in home', 'Entire bungalow',
       'Private room in tiny home', 'Private room in guesthouse',
       'Entire condo', 'Room in hotel', 'Private room in rental unit',
       'Entire guesthouse', 'Entire home', 'Entire rental unit',
       'Entire guest suite', 'Private room in loft', 'Entire townhouse',
       'Entire cottage', 'Entire villa', 'Private room',
       'Shared room in rental unit', 'Room in boutique hotel',
       'Private room in townhouse', 'Private room in hostel', 'Camper/RV',
       'Private room in bed and breakfast', 'Farm stay', 'Entire cabin',
       'Entire loft', 'Private room in farm stay',
       'Private room in condo', 'Private room in bungalow',
       'Private room in guest suite', 'Shared room in home',
       'Shared room in hostel', 'Shared room in villa', 'Entire place',
       'Room in aparthotel', 'Private room in treehouse', 'Treehouse',
       'Yurt', 'Private room in villa', 'Private room in cottage',
       'Private room in c

In [9]:
## removing attributes whch have no value to us
### keeping longitude and latitude in case we wanted to do a sort of heat map for airbnb hotspots for the exceptional points, if not we can remove
### neighborhood, neighborhood_cleansed, and neighborhood_group_cleansed variations of same, but cleansed seems to be better to use, I'm guessing it has imputed values
### for some reason, all listings have NaN for bathrooms, so removing it
for col in [
    'listing_url','scrape_id','last_scraped','description','neighborhood_overview','picture_url','host_url','host_about','host_response_time','host_response_rate','host_acceptance_rate',
    'host_thumbnail_url','host_picture_url','host_verifications','host_has_profile_pic','bathroom_text','host_listings_count','host_neighbourhood','bathrooms','minimum_minimum_nights',
    'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','availability_30','availability_60',
    'availability_90','availability_365','calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d','review_scores_accuracy','review_scores_communication','review_scores_cleanliness',
    'review_scores_checkin','review_scores_value','review_scores_location','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms','reviews_per_month','neighbourhood','neighbourhood_group_cleansed', 'first_review','last_review'
]:
    if col in df:
        del df[col]

In [10]:
df.head()

,id,name,host_id,host_name,host_since,host_location,host_is_superhost,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,...,amenities,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,review_scores_rating,license,instant_bookable,calculated_host_listings_count
0,45392,Cute Home in Mount Washington,201514,Olivia & Alexey,2010-08-14,"Los Angeles, California, United States",f,1.0,t,Mount Washington,...,"[""Hangers"", ""Free parking on premises"", ""Essen...",$60.00,30,30,t,27,4.88,NaN,f,1
1,45417,Silver Lake Cottage Oasis- Private Terrace- Th...,50231247,Tim,2015-11-30,"Camarillo, California, United States",t,8.0,t,Silver Lake,...,"[""Bathtub"", ""TV"", ""Iron"", ""Pack \u2019n play/T...",$135.00,30,180,t,154,4.91,NaN,f,4
2,5728,Tiny Home in Artistic Oasis near Venice and LAX,9171,Sanni,2009-03-05,"Los Angeles, California, United States",t,8.0,t,Del Rey,...,"[""Hangers"", ""Free parking on premises"", ""Essen...",$50.00,30,1125,t,314,4.79,HSR19-002149,f,3
3,5729,Zen Room with Floating Bed near Venice and LAX,9171,Sanni,2009-03-05,"Los Angeles, California, United States",t,8.0,t,Del Rey,...,"[""Hangers"", ""Free parking on premises"", ""Essen...",$65.00,30,1125,t,236,4.77,HSR19-002149,f,3
4,109,Amazing bright elegant condo park front *UPGRA...,521,Paolo,2008-06-27,"San Francisco, California, United States",f,1.0,t,Culver City,...,"[""Hangers"", ""Pool"", ""Free parking on premises""...",$115.00,30,730,t,2,4.00,NaN,f,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              42041 non-null  int64  
 1   name                            42039 non-null  object 
 2   host_id                         42041 non-null  int64  
 3   host_name                       41858 non-null  object 
 4   host_since                      41858 non-null  object 
 5   host_location                   41780 non-null  object 
 6   host_is_superhost               41858 non-null  object 
 7   host_total_listings_count       41858 non-null  float64
 8   host_identity_verified          41858 non-null  object 
 9   neighbourhood_cleansed          42041 non-null  object 
 10  latitude                        42041 non-null  float64
 11  longitude                       42041 non-null  float64
 12  property_type                   

In [12]:
df.bathrooms_text.unique()

array(['1 private bath', '1 bath', '1 shared bath', '2 baths',
       '1.5 shared baths', nan, '1.5 baths', '3 baths', '11 shared baths',
       '3.5 baths', '2 shared baths', '4 baths', '0 baths', '2.5 baths',
       '4.5 baths', '0 shared baths', '8 shared baths', '8 baths',
       '8.5 shared baths', '2.5 shared baths', '3.5 shared baths',
       '5 baths', '3 shared baths', '11.5 shared baths', '6.5 baths',
       '4.5 shared baths', '7.5 baths', 'Half-bath', '5.5 baths',
       'Shared half-bath', '8.5 baths', '11 baths', '5 shared baths',
       '6 baths', '7 baths', '10 baths', 'Private half-bath', '9 baths',
       '12.5 baths', '9.5 baths', '4 shared baths', '11.5 baths',
       '10.5 baths', '13 baths', '5.5 shared baths', '13.5 baths',
       '25 baths', '12 baths', '15.5 baths', '21 baths', '18 baths'],
      dtype=object)

### Business Understanding

### Data Meaning Type

### Data Quality

### Simple Statistics

### Visualize Attributes

### Explore Joint Attributes

### Explore Attributes and Class

### New Features

### Exceptional Work